# RAG-Based News Summarization Project

## Overview
This Jupyter Notebook implements a Retrieval-Augmented Generation (RAG) system for summarizing news articles from the Hugging Face `cc_news` dataset. The project demonstrates expertise in Generative AI, vector embeddings, and evaluation metrics, suitable for showcasing on a resume.

**Components**:
- **Data Loading**: Fetches and preprocesses news articles.
- **Vector Store**: Creates a FAISS index with Sentence Transformer embeddings.
- **RAG Pipeline**: Retrieves relevant articles and generates summaries using BART.
- **Evaluation**: Assesses retrieval (Precision@k, Recall@k, MRR) and generation (ROUGE) performance.

## Setup
Install required dependencies and import libraries. Run this cell first to set up the environment.

In [ ]:
# !pip install datasets==2.21.0 sentence-transformers==3.1.1 faiss-cpu==1.9.0 langchain==0.3.0 langchain_community==0.3.0 transformers==4.44.2 torch==2.4.1 pandas==2.2.3 numpy==1.26.4 rouge_score==0.1.2

# import logging
# import numpy as np
# import pandas as pd
# from datasets import load_dataset
# from sentence_transformers import SentenceTransformer # pre-trained models for text embeddings
# import faiss # for efficient similarity search
# import pickle
# from langchain.llms import HuggingFacePipeline # perform inference with HuggingFace models
# from langchain.prompts import PromptTemplate # create prompts for LLMs
# from langchain.chains import LLMChain # chain together LLM calls
# from transformers import pipeline
# from rouge_score import rouge_scorer
# from typing import List, Dict, Any


# Setup
# Install required dependencies and import libraries. Run this cell first.
# Set OpenAI API key for GPT-3.5-Turbo (optional for BART).

# !pip install datasets==2.21.0 sentence-transformers==3.1.1 faiss-cpu==1.9.0 langchain==0.3.0 langchain_community==0.3.0 langchain_openai==0.3.0 transformers==4.44.2 torch==2.4.1 pandas==2.2.3 numpy==1.26.4 rouge_score==0.1.2

import logging
import numpy as np
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import pickle
from langchain_openai import ChatOpenAI
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import pipeline
from rouge_score import rouge_scorer
from typing import List, Dict, Any
import os

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set OpenAI API key (required for model_type='openai')
# Option 1: Set environment variable (recommended)
os.environ["OPENAI_API_KEY"] = "my_test_API_key"
# Option 2: Input key directly (less secure, for testing)
try:
    if not os.environ.get("OPENAI_API_KEY"):
        OPENAI_API_KEY = input("Enter your OpenAI API key (or press Enter to skip for BART): ")
        if OPENAI_API_KEY:
            os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
except Exception as e:
    logger.warning(f"OpenAI API key setup skipped: {e}")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Data Loading
Fetch and preprocess news articles from the `cc_news` dataset on Hugging Face. This module loads a sample of articles and cleans the text for embedding and summarization.

In [9]:
class NewsDataLoader:
    """Loads and preprocesses news articles from Hugging Face dataset."""
    
    def __init__(self, dataset_name="cc_news", sample_size=100):
        self.dataset_name = dataset_name
        self.sample_size = sample_size
        self.data = None
    
    def load_data(self):
        """Load dataset from Hugging Face and sample articles."""
        try:
            logger.info(f"Loading dataset: {self.dataset_name}")
            dataset = load_dataset(self.dataset_name, split="train")
            self.data = dataset.to_pandas().sample(n=min(self.sample_size, len(dataset)), random_state=42)
            print(f"Sampled {len(self.data)} articles from the dataset")
            logger.info(f"Loaded {len(self.data)} articles")
        except Exception as e:
            logger.error(f"Error loading data: {e}")
            raise
    
    def preprocess_data(self):
        """Clean and preprocess text data."""
        if self.data is None:
            raise ValueError("Data not loaded. Call load_data() first.")
        
        logger.info("Preprocessing data")
        self.data = self.data.dropna(subset=['text'])
        self.data['text'] = self.data['text'].str.strip().str.replace(r'\s+', ' ', regex=True) # Remove extra whitespace, duplicate spaces
        self.data = self.data[self.data['text'].str.len() > 100]
        logger.info(f"Preprocessed {len(self.data)} articles")
    
    def get_articles(self):
        """Return preprocessed articles."""
        if self.data is None:
            raise ValueError("Data not loaded or preprocessed.")
        return self.data[['title', 'text']].to_dict('records')

# Load and preprocess data
loader = NewsDataLoader(sample_size=200)
loader.load_data()
loader.preprocess_data()
articles = loader.get_articles()
print(f"Sample article title: {articles[0]['title']}")


INFO:__main__:Loading dataset: cc_news
INFO:__main__:Loaded 200 articles
INFO:__main__:Preprocessing data
INFO:__main__:Preprocessed 198 articles


Sampled 200 articles from the dataset
Sample article title: ‘Indexpo’ in the city from June 16 to 18


In [10]:
print(f"Sample article title: {articles[15]['title']}")
print(f"Sample article text: {articles[15]['text']}")

Sample article title: Urban and Small Stream Flood Advisory until 4:15PM CDT for western McHenry County and Boone County in Illinois
Sample article text: × Urban and Small Stream Flood Advisory until 4:15PM CDT for western McHenry County and Boone County in Illinois * Urban and Small Stream Flood Advisory for… Western McHenry County in northeastern Illinois… Boone County in north central Illinois… * Until 415 PM CDT * At 110 PM CDT, Doppler radar indicated heavy rain due to thunderstorms. This will cause urban and small stream flooding in the advisory area. * Some locations that will experience flooding include… Belvidere, Woodstock, Harvard, Marengo, Poplar Grove, Capron, Hebron, Timberlane, Greenwood, Caledonia and Union. * Minor flooding was reported by law enforcement in Belvidere around 100 PM CDT.


## Vector Store
Create a FAISS vector store using Sentence Transformer embeddings for semantic search. This module enables efficient retrieval of relevant articles.

In [11]:
class VectorStore:
    """Manages creation and querying of FAISS vector store for news articles."""
    
    def __init__(self, model_name="all-MiniLM-L6-v2", index_path="faiss_index.bin", metadata_path="metadata.pkl"):
        self.model = SentenceTransformer(model_name)
        self.index_path = index_path
        self.metadata_path = metadata_path
        self.index = None
        self.metadata = None
    
    def create_index(self, articles):
        """Create FAISS index from article texts."""
        
        logger.info("Creating FAISS index")
        texts = [article['text'] for article in articles] # Extract text from articles
        embeddings = self.model.encode(texts, show_progress_bar=True) # Generate embeddings for texts
        dimension = embeddings.shape[1] # Get embedding dimension
        self.index = faiss.IndexFlatL2(dimension) # Create FAISS index for L2 distance
        self.index.add(embeddings.astype(np.float32)) # Add embeddings to FAISS index
        self.metadata = articles # Store metadata for articles
        logger.info(f"Indexed {len(articles)} articles")
    
    def save_index(self):
        """Save FAISS index and metadata to disk."""

        if self.index is None or self.metadata is None:
            raise ValueError("Index or metadata not initialized.")
        faiss.write_index(self.index, self.index_path) # Save FAISS index to file
        with open(self.metadata_path, 'wb') as f:
            pickle.dump(self.metadata, f)
        logger.info(f"Saved index to {self.index_path} and metadata to {self.metadata_path}")
    
    def load_index(self):
        """Load FAISS index and metadata from disk."""
        try:
            self.index = faiss.read_index(self.index_path)
            with open(self.metadata_path, 'rb') as f:
                self.metadata = pickle.load(f)
            logger.info(f"Loaded index from {self.index_path} and metadata from {self.metadata_path}")
        except Exception as e:
            logger.error(f"Error loading index: {e}")
            raise
    
    def search(self, query, k=5):
        """Search for top-k relevant articles based on query."""
        if self.index is None:
            raise ValueError("Index not loaded or created.")
        query_embedding = self.model.encode([query])[0]
        distances, indices = self.index.search(np.array([query_embedding]).astype(np.float32), k)
        results = [self.metadata[i] for i in indices[0]]
        return results

# Create and save vector store
store = VectorStore()
store.create_index(articles)
store.save_index()
store.load_index()
results = store.search("Art", k=5) # Search for articles related in vector store
print(f"Search results: {[article['title'] for article in results]}")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
/Users/user/Desktop/GitHub/GenAI_RAG_News_Summary/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:__main__:Creating FAISS index
Batches: 100%|██████████| 7/7 [00:01<00:00,  3.71it/s]
INFO:__main__:Indexed 198 articles
INFO:__main__:Saved index to faiss_index.bin and metadata to metadata.pkl
INFO:__main__:Loaded index from faiss_index.bin and metadata from metadata.pkl
Batches: 100%|██████████| 1/1 [00:00<00:00, 23.03it/s]

Search results: ['There are many youtuber’s just who make video', 'This Guy Goes To Hilariously Impressive Lengths To Take Instagrams Of His Girlfriend', 'Showcase your Zine', 'It Brits Celebrated Mrs. Alice x Misela at Annabel’s', 'Insomniac Magazine']


In [12]:
results = store.search("America", k=5) # Search for articles related in vector store
print(f"Search results: {[article['title'] for article in results]}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 75.46it/s]

Search results: ['Fox Paid $400 Million for 2018 World Cup Broadcast Rights, Then Team USA Got Eliminated', 'Let’s Talk About Paul Ryan’s Scorching Duplicity on Gun Violence, Mental Health, and, Well, Everything', 'Insomniac Magazine', "In drug crisis hotbed, hoping for action on Trump's words", 'Top US Military Officer Warns North Korea That US Military Ready']


## RAG Pipeline
Implement the RAG pipeline to retrieve relevant articles and generate summaries using a BART model from Hugging Face.

In [13]:
# class RAGPipeline:
#     """Implements Retrieval-Augmented Generation for news summarization."""
    
#     def __init__(self, vector_store, model_name="openai/gpt-3.5-turbo"): # Hugging Face model for summarization, can be changed to any other model, example: "t5-base", "openai/gpt-3.5-turbo"
#         self.vector_store = vector_store
#         self.model_name = model_name
#         self.llm = self._initialize_llm()
    
#     def _initialize_llm(self):
#         """Initialize Hugging Face model for summarization."""
#         logger.info(f"Initializing LLM: {self.model_name}")
#         hf_pipeline = pipeline("summarization", model=self.model_name, device=-1)
#         llm = HuggingFacePipeline(pipeline=hf_pipeline)
#         return llm
    
#     def create_prompt(self, query, retrieved_articles):
#         """Create prompt for summarization based on query and retrieved articles."""
#         context = "\n".join([f"Article: {article['text'][:1000]}" for article in retrieved_articles]) # Limit to first 1000 characters for brevity
#         template = """
#         Summarize the following news articles in 2-3 sentences, focusing on the topic: {query}.
#         Context:
#         {context}
#         """
#         prompt = PromptTemplate(template=template, input_variables=["query", "context"])
#         return prompt.format(query=query, context=context)
    
#     def generate_summary(self, query, k=3):
#         """Generate summary for query using RAG."""
#         logger.info(f"Generating summary for query: {query}")
#         retrieved_articles = self.vector_store.search(query, k=k) # Retrieve top-k articles
#         if not retrieved_articles:
#             logger.warning("No articles retrieved for the query.")
#             return {"query": query, "summary": "No relevant articles found.", "retrieved_articles": []}
#         logger.info(f"Retrieved {len(retrieved_articles)} articles for summarization")

#         # Create prompt and run summarization
#         prompt = self.create_prompt(query, retrieved_articles)  
#         chain = LLMChain(llm=self.llm, prompt=PromptTemplate(template="{text}", input_variables=["text"])) # Use LLMChain for summarization
#         summary = chain.run(text=prompt)
#         return {
#             "query": query,
#             "summary": summary,
#             "retrieved_articles": [article['title'] for article in retrieved_articles]
#         }

# # Initialize RAG pipeline
# rag = RAGPipeline(store)
# result = rag.generate_summary("USA", k=5)
# print(f"Summary: {result['summary']}")
# print(f"Retrieved articles: {result['retrieved_articles']}")

In [15]:
# RAG Pipeline
# Implement the RAG pipeline to retrieve relevant articles and generate summaries using either OpenAI GPT-3.5-Turbo or Facebook BART.

class RAGPipeline:
    """Implements Retrieval-Augmented Generation for news summarization."""
    
    def __init__(self, vector_store, model_type="bart", openai_model="gpt-3.5-turbo", bart_model="facebook/bart-large-cnn"):
        """
        Initialize RAG pipeline with specified model type.
        Args:
            vector_store (VectorStore): Instance for retrieval.
            model_type (str): 'openai' for GPT-3.5-Turbo or 'bart' for BART.
            openai_model (str): OpenAI model name (if model_type='openai').
            bart_model (str): BART model name (if model_type='bart').
        """
        self.vector_store = vector_store
        self.model_type = model_type.lower()
        self.openai_model = openai_model
        self.bart_model = bart_model
        self.llm = self._initialize_llm()
    
    def _initialize_llm(self):
        """Initialize LLM based on model type."""
        logger.info(f"Initializing LLM: {self.model_type}")
        try:
            if self.model_type == "openai":
                if not os.environ.get("OPENAI_API_KEY"):
                    raise ValueError("OPENAI_API_KEY environment variable not set.")
                llm = ChatOpenAI(model_name=self.openai_model, temperature=0.7)
            elif self.model_type == "bart":
                hf_pipeline = pipeline("summarization", model=self.bart_model, device=-1)
                llm = HuggingFacePipeline(pipeline=hf_pipeline)
            else:
                raise ValueError("model_type must be 'openai' or 'bart'.")
            return llm
        except Exception as e:
            logger.error(f"Error initializing {self.model_type} model: {e}")
            raise
    
    def create_prompt(self, query, retrieved_articles):
        """Create prompt for summarization based on query and retrieved articles."""
        context = "\n".join([f"Article: {article['text'][:1000]}" for article in retrieved_articles])
        if self.model_type == "openai":
            template = """
            You are a helpful assistant tasked with summarizing news articles.
            Summarize the following articles in 2-3 sentences, focusing on the topic: {query}.
            Ensure the summary is concise, relevant, and factually accurate.
            Context:
            {context}
            """
        else:  # BART
            template = """
            Summarize the following news articles in 2-3 sentences, focusing on the topic: {query}.
            Context:
            {context}
            """
        prompt = PromptTemplate(template=template, input_variables=["query", "context"])
        return prompt.format(query=query, context=context)
    
    def generate_summary(self, query, k=3):
        """Generate summary for query using RAG."""
        logger.info(f"Generating summary for query: {query}")
        retrieved_articles = self.vector_store.search(query, k=k)
        prompt = self.create_prompt(query, retrieved_articles)
        chain = LLMChain(llm=self.llm, prompt=PromptTemplate(template="{text}", input_variables=["text"]))
        summary = chain.run(text=prompt)
        return {
            "query": query,
            "summary": summary,
            "retrieved_articles": [article['title'] for article in retrieved_articles]
        }

# Test RAG pipeline with both models
# BART (default)
rag_bart = RAGPipeline(store, model_type="bart")
result_bart = rag_bart.generate_summary("USA", k=2)
print(f"BART Summary: {result_bart['summary']}")
print(f"BART Retrieved articles: {result_bart['retrieved_articles']}")

# OpenAI 
try:
    rag_openai = RAGPipeline(store, model_type="openai")
    result_openai = rag_openai.generate_summary("USA", k=2)
    print(f"OpenAI Summary: {result_openai['summary']}")
    print(f"OpenAI Retrieved articles: {result_openai['retrieved_articles']}")
except Exception as e:
    print(f"OpenAI test skipped: {e}")

INFO:__main__:Initializing LLM: bart
/Users/user/Desktop/GitHub/GenAI_RAG_News_Summary/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:__main__:Generating summary for query: USA
Batches: 100%|██████████| 1/1 [00:02<00:00,  2.70s/it]
INFO:__main__:Initializing LLM: openai
INFO:__main__:Generating summary for query: USA


BART Summary: Fox jumped into a bidding war with ESPN to win the English language broadcast rights for the 2018 World Cup. Fox topped ESPN in the bidding war by $200 million bringing its final costs to a whopping $400 million for the rights. The cost may have seemed like a good deal since the U.S. soccer team hadn’t missed a World Cup since 1986. However, now that Team USA was knocked out in the early stages of the tournament, it looks like a disastrous decision.
BART Retrieved articles: ['Fox Paid $400 Million for 2018 World Cup Broadcast Rights, Then Team USA Got Eliminated', 'Top US Military Officer Warns North Korea That US Military Ready']


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.76it/s]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.433046 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.815848 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"


OpenAI test skipped: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


## Evaluation
Evaluate the RAG system's retrieval (Precision@k, Recall@k, MRR) and generation (ROUGE) performance using a small evaluation dataset.

In [ ]:
class Evaluator:
    """Evaluates retrieval and generation performance of the RAG system."""
    
    def __init__(self, vector_store, rag_pipeline):
        self.vector_store = vector_store
        self.rag_pipeline = rag_pipeline
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    def evaluate_retrieval(self, queries: List[str], ground_truth: Dict[str, List[int]], k: int = 3) -> Dict[str, float]:
        precisions, recalls, mrrs = [], [], []
        
        for query in queries:
            retrieved_articles = self.vector_store.search(query, k=k)
            retrieved_indices = [self.vector_store.metadata.index(article) for article in retrieved_articles]
            relevant_indices = ground_truth.get(query, [])
            
            relevant_retrieved = len(set(retrieved_indices) & set(relevant_indices))
            precision = relevant_retrieved / k if k > 0 else 0
            precisions.append(precision)
            
            recall = relevant_retrieved / len(relevant_indices) if relevant_indices else 0
            recalls.append(recall)
            
            mrr = 0
            for rank, idx in enumerate(retrieved_indices, 1):
                if idx in relevant_indices:
                    mrr = 1 / rank
                    break
            mrrs.append(mrr)
        
        return {
            "precision@k": np.mean(precisions),
            "recall@k": np.mean(recalls),
            "mrr": np.mean(mrrs)
        }
    
    def evaluate_generation(self, queries: List[str], reference_summaries: Dict[str, str], k: int = 3) -> Dict[str, float]:
        rouge_scores = {"rouge1": [], "rouge2": [], "rougeL": []}
        
        for query in queries:
            result = self.rag_pipeline.generate_summary(query, k=k)
            generated_summary = result["summary"]
            reference_summary = reference_summaries.get(query, "")
            
            if reference_summary:
                scores = self.rouge_scorer.score(reference_summary, generated_summary)
                for metric in rouge_scores:
                    rouge_scores[metric].append(scores[metric].fmeasure)
        
        return {
            "rouge1": np.mean(rouge_scores["rouge1"]) if rouge_scores["rouge1"] else 0,
            "rouge2": np.mean(rouge_scores["rouge2"]) if rouge_scores["rouge2"] else 0,
            "rougeL": np.mean(rouge_scores["rougeL"]) if rouge_scores["rougeL"] else 0
        }
    
    def evaluate_end_to_end(self, queries: List[str], ground_truth: Dict[str, List[int]], reference_summaries: Dict[str, str], k: int = 3) -> Dict[str, float]:
        retrieval_metrics = self.evaluate_retrieval(queries, ground_truth, k)
        generation_metrics = self.evaluate_generation(queries, reference_summaries, k)
        return {**retrieval_metrics, **generation_metrics}

# Setup evaluation data (examples)
queries = ["UK", "USA"]
ground_truth = {
    "UK": [9, 13],  
    "USA": [4, 19]
}
reference_summaries = {
    "UK": "Ross Kemp reacts BRILLIANTLY to England's win over Colombia",
    "USA": "Donald Trump can't hide behind patriotism if he won't condemn neo-Nazi thugs"
}

# Run evaluation
evaluator = Evaluator(store, rag_bart)
metrics = evaluator.evaluate_end_to_end(queries, ground_truth, reference_summaries, k=3)
print("Evaluation Metrics:", metrics)

INFO:absl:Using default tokenizer.
Batches: 100%|██████████| 1/1 [00:00<00:00, 84.10it/s]
INFO:__main__:Generating summary for query: UK
Batches: 100%|██████████| 1/1 [00:00<00:00, 63.04it/s]
INFO:__main__:Generating summary for query: USA
Batches: 100%|██████████| 1/1 [00:00<00:00,  2.60it/s]


Evaluation Metrics: {'precision@k': 0.16666666666666666, 'recall@k': 0.25, 'mrr': 0.25, 'rouge1': 0.026765188834154352, 'rouge2': 0.0, 'rougeL': 0.026765188834154352}


### Interpretations
- **Precision@k = 0.167 (16.7%)**:
  - Only 16.7% of the top-3 retrieved articles are relevant to the query.
  - Poor retrieval accuracy; many retrieved articles are irrelevant (limited dataset use can be the reason).

- **Recall@k = 0.25 (25%)**:
  - *25% of all relevant articles are retrieved in the top-3.
  -  Misses most relevant articles, likely due to small dataset.

- **Mean Reciprocal Rank (MRR) = 0.25**:
  - First relevant article appears around rank 4 on average (1/0.25).
  - Relevant articles are ranked low, reducing retrieval effectiveness.

- **ROUGE-1 = 0.027 (2.7%)**:
  - 2.7% overlap of single words between generated and reference summaries (also likely due to small dataset).
  - Minimal content similarity, possibly due to irrelevant retrieved articles or paraphrasing.

- **ROUGE-2 = 0.0 (0%)**:
  - No overlap of two-word phrases between generated and reference summaries.
  - No shared phrases, indicating highly abstractive summaries or mismatched references.

- **ROUGE-L = 0.027 (2.7%)**:
  - 2.7% overlap in longest common subsequence, measuring structural similarity.
  - Poor structural alignment, likely due to same issues as ROUGE-1.

- **Overall**:
  - **Retrieval**: Weak performance (low precision, recall, MRR) suggests issues with embeddings or dataset size.
  - **Generation**: Very low ROUGE scores indicate summaries don’t match references, likely due to poor retrieval or reference quality.
  - **Next Steps**: Increase the dataset size, refine the `ground_truth` and the reference summaries, maybe also upgrade the embedding model (e.g., `all-mpnet-base-v2`).

## Interactive Exploration
Test the RAG system by entering custom queries. Run this cell to try different topics and inspect the results.

In [20]:
def interactive_query():
    """Run interactive query loop for testing the RAG system."""
    while True:
        query = input("Enter a query (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break
        result = rag_openai.generate_summary(query, k=3)
        print(f"\nSummary: {result['summary']}")
        print(f"Retrieved articles: {result['retrieved_articles']}")

interactive_query()

INFO:__main__:Generating summary for query: Donald Trump
Batches: 100%|██████████| 1/1 [00:02<00:00,  2.94s/it]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.382580 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.920185 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Visualize Articles
Display a sample of article titles and their texts to help create evaluation data (e.g., ground truth indices).

In [21]:
# Display first 5 articles for inspection
for i, article in enumerate(articles[:20]):
    print(f"Index {i}: {article['title']}")
    print(f"Text (first 200 chars): {article['text'][:200]}\n")

Index 0: ‘Indexpo’ in the city from June 16 to 18
Text (first 200 chars): Nashik : Indore Infoline Pvt. Ltd has organised an industrial expo ‘Indexpo’ at Thakkar’s Dome, ABB Circle, Nashik from June 16 to 18, informed managing director Rajkumar Agrawal in a media briefing y

Index 1: Quit notice: We are monitoring situation in Northern Nigeria - South-East Governors
Text (first 200 chars): South-East Governors on Monday re-assured Ndigbo residing in different parts of the country of their safety. The Governors, who met in Enugu said they were in constant touch with the Northern Governor

Index 2: Two teens charged, accused of leading police on chase in reporte - | WBTV Charlotte
Text (first 200 chars): The two teenagers that were arrested in connection to a vehicle pursuit involving a stolen vehicle Wednesday have been identified. Charlotte-Mecklenburg Police identified the two teens as 17-year-old 

Index 3: Golf: 'Green Mile' strewn with PGA victims at Quail Hollow
Text (first 200 ch

## Notes
- **Evaluation Data**: The `ground_truth` and `reference_summaries` are placeholders. Inspect articles using the visualization cell above to assign correct indices and write reference summaries.
- **Performance**: The `facebook/bart-large-cnn` model is CPU-intensive. For faster execution, one can try `distilbart-cnn-6-6` or use a GPU with `torch` CUDA support.
- **Future Extensions**:
  - Adding a web UI with Streamlit for better interactivity.
  - Using a larger dataset or CNN/DailyMail for pre-annotated summaries.
  - Implementing additional metrics like BLEU or human evaluation.